In [30]:
"""Note: Several of these functions came from a comment on StackOverflow
But I adapted to Python 3, so any mistakes are probably mine :/
https://stackoverflow.com/questions/28595958/creating-trackbars-to-scroll-large-image-in-opencv-python/33293804#33293804"""

import numpy as np
import PIL
from PIL import Image
import argparse
import cv2
from pprint import pprint

def onLeftClickFunction(y,x):
    global clickCounter
    global nodes
    
    #put a dot where we clicked
    clickSpot = (int(x),int(y))
    radius = 4
    color = (255,0,0)
    cv2.circle(image, clickSpot, radius, color, thickness=-1, lineType=8, shift=0)
      
    #Add node to array
    nodes.append(clickSpot)
    
    #print out the list of nodes
    #print("Node ", clickCounter, ": (", x, ",", y, ")")
    clickCounter+=1

    
class PanZoomWindow(object):
    """ Controls an OpenCV window. Registers a mouse listener so that:
        1. right-dragging up/down zooms in/out
        2. right-clicking re-centers
        3. trackbars scroll vertically and horizontally 
    You can open multiple windows at once if you specify different window names.
    You can pass in an onLeftClickFunction, and when the user left-clicks, this 
    will call onLeftClickFunction(y,x), with y,x in original image coordinates."""
    def __init__(self, img, windowName = 'PanZoomWindow', onLeftClickFunction = onLeftClickFunction):
        self.WINDOW_NAME = windowName
        self.H_TRACKBAR_NAME = 'x'
        self.V_TRACKBAR_NAME = 'y'
        self.img = img
        self.onLeftClickFunction = onLeftClickFunction
        self.TRACKBAR_TICKS = 1000
        self.panAndZoomState = PanAndZoomState(img.shape, self)
        self.lButtonDownLoc = None
        self.mButtonDownLoc = None
        self.rButtonDownLoc = None
        cv2.namedWindow(self.WINDOW_NAME, cv2.WINDOW_NORMAL)
        self.redrawImage()
        cv2.setMouseCallback(self.WINDOW_NAME, self.onMouse)
        cv2.createTrackbar(self.H_TRACKBAR_NAME, self.WINDOW_NAME, 0, self.TRACKBAR_TICKS, self.onHTrackbarMove)
        cv2.createTrackbar(self.V_TRACKBAR_NAME, self.WINDOW_NAME, 0, self.TRACKBAR_TICKS, self.onVTrackbarMove)
    def onMouse(self,event, x,y,_ignore1,_ignore2):
        """ Responds to mouse events within the window. 
        The x and y are pixel coordinates in the image currently being displayed.
        If the user has zoomed in, the image being displayed is a sub-region, so you'll need to
        add self.panAndZoomState.ul to get the coordinates in the full image."""
        if event == cv2.EVENT_MOUSEMOVE:
            return
        elif event == cv2.EVENT_RBUTTONDOWN:
            #record where the user started to right-drag
            self.mButtonDownLoc = np.array([y,x])
        elif event == cv2.EVENT_RBUTTONUP and self.mButtonDownLoc is not None:
            #the user just finished right-dragging
            dy = y - self.mButtonDownLoc[0]
            pixelsPerDoubling = 0.2*self.panAndZoomState.shape[0] #lower = zoom more
            changeFactor = (1.0+abs(dy)/pixelsPerDoubling)
            changeFactor = min(max(1.0,changeFactor),5.0)
            if changeFactor < 1.05:
                dy = 0 #this was a click, not a draw. So don't zoom, just re-center.
            if dy > 0: #moved down, so zoom out.
                zoomInFactor = 1.0/changeFactor
            else:
                zoomInFactor = changeFactor
            #print("redrawing at: (",self.mButtonDownLoc[0], ",", self.mButtonDownLoc[1], "). New zoom factor is: ",  zoomInFactor )
            self.panAndZoomState.zoom(self.mButtonDownLoc[0], self.mButtonDownLoc[1], zoomInFactor)
        elif event == cv2.EVENT_LBUTTONDOWN:
            #the user pressed the left button. 
            coordsInDisplayedImage = np.array([y,x])
            if np.any(coordsInDisplayedImage < 0) or np.any(coordsInDisplayedImage > self.panAndZoomState.shape[:2]):
                print ("you clicked outside the image area")
            else:
                #print ("you clicked on",coordsInDisplayedImage,"within the zoomed rectangle")
                coordsInFullImage = self.panAndZoomState.ul + coordsInDisplayedImage
                #print ("this is",coordsInFullImage,"in the actual image")
                #print ("this pixel holds ",self.img[int(coordsInFullImage[0]),int(coordsInFullImage[1])])
                if self.onLeftClickFunction is not None:
                    self.onLeftClickFunction(coordsInFullImage[0],coordsInFullImage[1])
                self.panAndZoomState._fixBoundsAndDraw()
        #you can handle other mouse click events here
    def onVTrackbarMove(self,tickPosition):
        self.panAndZoomState.setYFractionOffset(float(tickPosition)/self.TRACKBAR_TICKS)
    def onHTrackbarMove(self,tickPosition):
        self.panAndZoomState.setXFractionOffset(float(tickPosition)/self.TRACKBAR_TICKS)
    def redrawImage(self):
        pzs = self.panAndZoomState
        #print(vars(pzs))
        xMin = int(pzs.ul[0])
        xMax = int(pzs.ul[0]+int(pzs.shape[0]))
        yMin = int(pzs.ul[1])
        yMax = int(pzs.ul[1]+int(pzs.shape[1]))
        #print(xMin)
        #print(xMax)
        #print(yMin)
        #print(yMax)
        cv2.imshow(self.WINDOW_NAME, self.img[xMin:xMax, yMin:yMax])

class PanAndZoomState(object):
    """ Tracks the currently-shown rectangle of the image.
    Does the math to adjust this rectangle to pan and zoom."""
    MIN_SHAPE = np.array([50,50])
    def __init__(self, imShape, parentWindow):
        self.ul = np.array([0,0]) #upper left of the zoomed rectangle (expressed as y,x)
        self.imShape = np.array(imShape[0:2])
        self.shape = self.imShape #current dimensions of rectangle
        self.parentWindow = parentWindow
    def zoom(self,relativeCy,relativeCx,zoomInFactor):
        self.shape = (self.shape.astype(np.float)/zoomInFactor).astype(np.int)
        #expands the view to a square shape if possible. (I don't know how to get the actual window aspect ratio)
        self.shape[:] = np.max(self.shape) 
        self.shape = np.maximum(PanAndZoomState.MIN_SHAPE,self.shape) #prevent zooming in too far
        c = self.ul+np.array([relativeCy,relativeCx])
        self.ul = c-self.shape/2
        self._fixBoundsAndDraw()
    def _fixBoundsAndDraw(self):
        """ Ensures we didn't scroll/zoom outside the image. 
        Then draws the currently-shown rectangle of the image."""
        #print "in self.ul:",self.ul, "shape:",self.shape
        self.ul = np.maximum(0,np.minimum(self.ul, self.imShape-self.shape))
        self.shape = np.minimum(np.maximum(PanAndZoomState.MIN_SHAPE,self.shape), self.imShape-self.ul)
        #print "out self.ul:",self.ul, "shape:",self.shape
        yFraction = float(self.ul[0])/max(1,self.imShape[0]-self.shape[0])
        xFraction = float(self.ul[1])/max(1,self.imShape[1]-self.shape[1])
        cv2.setTrackbarPos(self.parentWindow.H_TRACKBAR_NAME, self.parentWindow.WINDOW_NAME,int(xFraction*self.parentWindow.TRACKBAR_TICKS))
        cv2.setTrackbarPos(self.parentWindow.V_TRACKBAR_NAME, self.parentWindow.WINDOW_NAME,int(yFraction*self.parentWindow.TRACKBAR_TICKS))
        self.parentWindow.redrawImage()
    def setYAbsoluteOffset(self,yPixel):
        self.ul[0] = min(max(0,yPixel), self.imShape[0]-self.shape[0])
        self._fixBoundsAndDraw()
    def setXAbsoluteOffset(self,xPixel):
        self.ul[1] = min(max(0,xPixel), self.imShape[1]-self.shape[1])
        self._fixBoundsAndDraw()
    def setYFractionOffset(self,fraction):
        """ pans so the upper-left zoomed rectange is "fraction" of the way down the image."""
        self.ul[0] = int(round((self.imShape[0]-self.shape[0])*fraction))
        self._fixBoundsAndDraw()
    def setXFractionOffset(self,fraction):
        """ pans so the upper-left zoomed rectange is "fraction" of the way right on the image."""
        self.ul[1] = int(round((self.imShape[1]-self.shape[1])*fraction))
        self._fixBoundsAndDraw()

        
        
image = cv2.imread('C:/Users/cmj7g/Dropbox/VT/Long_Trail/all_maps_combined.png')

nodes = []
clickCounter = 0
nodeCount = 0

#Intialize by reading in the list of nodes we've previously identified
nodeFile = open("C:/Users/cmj7g/Dropbox/VT/Long_Trail/nodes.txt","r") 
thisNode = nodeFile.readline().strip()
while thisNode:
    thisNode = eval(thisNode)
    nodes.append(thisNode)
    cv2.circle(image, thisNode, radius=4, color=(255,0,0), thickness=-1, lineType=8, shift=0)
    
    
    #Add the node number
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = thisNode
    fontScale              = 0.5
    fontColor              = (255,255,255)
    lineType               = 2
    cv2.putText(image," " + str(nodeCount), bottomLeftCornerOfText, font, fontScale,fontColor,lineType)
    
    thisNode = nodeFile.readline().strip()
    nodeCount += 1
nodeFile.close()    


#Create the new window with the image and all nodes
window = PanZoomWindow(image, "test window")

key = -1
while key != ord('q') and key != 27: # 27 = escape key
    #the OpenCV window won't display until you call cv2.waitKey()
    key = cv2.waitKey(5) #User can press 'q' or ESC to exit.

#this all happens after the 'quit' command is received
#sort the nodes south to north to match the map
nodes.sort(key=lambda x: x[1], reverse=True)    
print(*nodes, sep = "\n")

#write out the latest list of nodes to the file
nodeFile = open("C:/Users/cmj7g/Dropbox/VT/Long_Trail/nodes.txt","w")
for node in nodes:
    nodeFile.write(str(node) + "\n")
nodeFile.close() 

#Save the current version of the image with all the existing nodes labelled
cv2.imwrite('C:/Users/cmj7g/Dropbox/VT/Long_Trail/full_map_with_nodes.png',image)

#close the image window
cv2.destroyAllWindows()

(848, 29298)
(553, 29291)
(1016, 28970)
(1051, 28893)
(1124, 28237)
(1167, 27959)
(1181, 27529)
(589, 27482)
(1117, 27261)
(910, 27114)
(1569, 27033)
(1623, 27007)
(1562, 26396)
(2402, 25400)
(2243, 25343)
(2604, 25315)
(2443, 25303)
(2790, 25224)
(3006, 25002)
(3039, 24895)
(2814, 24827)
(2656, 24796)
(2648, 24782)
(2282, 24778)
(3172, 24660)
(2407, 24203)
(2269, 24155)
(1976, 24094)
(2247, 23939)
(2601, 23530)
(2897, 22911)
(2676, 22553)
(2714, 22301)
(2716, 22266)
(2472, 22229)
(2617, 22224)
(2656, 22092)
(2695, 22060)
(2983, 21867)
(2853, 21618)
(2626, 21578)
(2682, 21498)
(2750, 21195)
(2753, 21138)
(2731, 21128)
(2439, 21036)
(2864, 20653)
(2847, 20631)
(2786, 20605)
(2916, 20589)
(2831, 20561)
(2855, 20560)
(2858, 20504)
(2936, 20502)
(3017, 19701)
(3235, 19625)
(3459, 19533)
(3477, 19495)
(3607, 19329)
(4123, 19220)
(3663, 19180)
(4343, 19153)
(3701, 19086)
(4099, 18933)
(3995, 18763)
(3975, 18563)
(3955, 18550)
(3939, 18534)
(3472, 18367)
(3896, 18196)
(3895, 18127)
(3848, 181